In [1]:
import os
os.chdir("../")



In [2]:
from collections import namedtuple
from pathlib import Path

DataIngestionConfig = namedtuple("DataIngestionConfig", [
    "root_dir",
    "source",
    "local_data_file",
    "unzip_dir"
])

In [3]:

from deepClassifier.utils import *
from deepClassifier.utils import read_yaml
from deepClassifier.constants import *

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH

    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories(["artifacts/data_ingestion"])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source = config.source,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config

In [5]:
#Data ingestion loading part
import os
from zipfile import ZipFile

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        

    def _get_updated_list_of_files(self, list_of_files):
        return [f for f in list_of_files if f.endswith(".png")and ("mask_weared_incorrect" in f or "with_mask" in f or "without_mask" in f)]

    def _preprocess(self, zf: ZipFile, f: str, working_dir: str):
        target_filepath = os.path.join(working_dir, f)
        if not os.path.exists(target_filepath):
            zf.extract(f, working_dir)
        
        if os.path.getsize(target_filepath) == 0:
            os.remove(target_filepath)

    def unzip_and_clean(self):
        with ZipFile(file=self.config.source, mode="r") as zf:
            list_of_files = zf.namelist()
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)
            for f in updated_list_of_files:
                self._preprocess(zf, f, self.config.unzip_dir)

In [15]:

config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion = DataIngestion(config=data_ingestion_config)
data_ingestion.unzip_and_clean()


In [6]:
config_filepath = CONFIG_FILE_PATH

In [7]:
config_filepath

WindowsPath('configs/config.yaml')

In [8]:
config = read_yaml(config_filepath)

In [9]:
config

ConfigBox({'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source': 'face-mask-detection.zip', 'local_data_file': 'artifacts/data_ingestion/face-mask-detection.zip', 'unzip_dir': 'artifacts/data_ingestion'}})

In [10]:
config = config.data_ingestion

In [11]:
create_directories([config.root_dir])

In [12]:
data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source = config.source,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )

In [13]:
data_ingestion_config

DataIngestionConfig(root_dir='artifacts/data_ingestion', source='face-mask-detection.zip', local_data_file='artifacts/data_ingestion/face-mask-detection.zip', unzip_dir='artifacts/data_ingestion')